**SCRAPPING**

Dans cette première étape, nous allons effectuer du scrapping sur le site Monster.fr

Celui-ci référence les emplois, le salaires, les zones sur une seule et même page.

Il n'est pas nécéssaire de changer de page pour afficher plus d'offres, les emplois s'actualisent automatiquement en scrollant en bas de page.

**Ce code a donc pour but de :**
- Se rendre sur le site Monster.fr, section Data scientist
- Scroller automatiquement en bas de page pendant X secondes (modifiable dans le code)
- Collecter les données et les ranger dans un tableau via pandas (Nom des posts, zones, salaires, mois en cours, années en cours)
- Créer un fichier Excel avec les données collectées

Vous pouvez run all directement

In [53]:
%pip install selenium

     ---------------------------------------- 0.0/118.7 kB ? eta -:--:--
     -------------------------------------- 118.7/118.7 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   --- ------------------------------------ 0.9/10.0 MB 18.7 MB/s eta 0:00:01
   ------- -------------------------------- 1.8/10.0 MB 23.4 MB/s eta 0:00:01
   -------------- ------------------------- 3.6/10.0 MB 28.5 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/10.0 MB 29.2 MB/s eta 0:00:01
   -------------------------- ------------- 6.6/10.0 MB 29.9 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/10.0 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------  9.9/10.0 MB 31.5 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 31.8 MB/s eta 0:00:01



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [72]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
from datetime import datetime

def extract_salary(salary_str):
    numbers = re.findall(r'\d+', salary_str)
    return int(''.join(numbers)) if numbers else 0

# Initialisation du pilote Selenium. La page étant sous JS, nous ne pouvons pas utiliser Beautifulsoup qui s'orriente sur du html uniquement 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Accès à la page de Monster.fr
# Monster, n'est pas un site protégé et est donc scrappable
url = "https://www.monster.fr/emploi/recherche?q=data+scientist&where=france&page=3&so=m.h.s"
driver.get(url)

# Scroll pendant 10 secondes jusqu'en bas de page pour charger plus d'offres afin de récupérer la data
# Possibilité d'augmenter le temps de scroll avec range pour avoir plus de données
for _ in range(60):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

# Attente supplémentaire pour le chargement du contenu au cas où
time.sleep(5)

# Collecte des données
data = []
today = datetime.today()

# Ajout des date afin de faire de l'analyse et pour tirer la donnée
# Mois actuel
current_month = today.strftime('%m') 

# Année actuelle 
current_year = today.strftime('%Y')
  
job_listings = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="svx_jobCard"]')
for job in job_listings:
    job_title_elements = job.find_elements(By.CSS_SELECTOR, 'a[data-testid="jobTitle"]')
    location_elements = job.find_elements(By.CSS_SELECTOR, 'span[data-testid="jobDetailLocation"]')
    salary_elements = job.find_elements(By.CSS_SELECTOR, 'span.sc-fKMpNL.dasswI.ds-tag-label')

    job_title = job_title_elements[0].text if job_title_elements else 'N/A'
    location = location_elements[0].text if location_elements else 'N/A'
    salary = extract_salary(salary_elements[0].text) if salary_elements else 0

    data.append([job_title, location, salary, current_month, current_year])

# Fermeture du navigateur
driver.quit()

# Création d'un DataFrame avec pandas
df = pd.DataFrame(data, columns=["Job Title", "Location", "Salary", "Month", "Year"])

# Suppression des lignes où le salaire est 0
df = df[df['Salary'] > 0]

# Enregistrement dans un fichier Excel
df.to_excel("DataJobs.xlsx", index=False)
